# Ollama function calling

First setup the notebook

In [1]:
#load "load.fsx"

open Newtonsoft.Json

open Informedica.OpenAI.Lib
open Ollama.Operators

## Create a tools list

The tools list contain the "tools", i.e. the functions that can be use by the LLM to return the answer.

In [4]:
let tools =
    {|
      city =  {|
        ``type`` =  "string"
        description = "The city to get the weather for"
      |}
      state = {|
        ``type`` =  "string"
        description = "The state to get the weather for"
      |}
    |}
    |> Ollama.Tool.create
        "get_weather"
        "Get the weather."
        ["city"; "state"]
    |> List.singleton

printfn $"## The tools object:\n{tools}"

## The tools object:
[{ type = "function"
  function =
   { name = "get_weather"
     description = "Get the weather."
     parameters =
      { type = "object"
        properties =
         { city = { description = "The city to get the weather for"
                    type = "string" }
           state = { description = "The state to get the weather for"
                     type = "string" } }
        required = ["city"; "state"] } } }]


## Run a question using the tools object

Note: function calling with ollama is (currently) really slow. Maybe because it tries to run the function that is actually not there?

In [3]:
"What is the weather in Seattle?"
|> Message.user
|> Ollama.extract
    tools
    "joefamous/firefunction-v1:q3_k"
    []
|> Async.RunSynchronously
|> function
    | Ok resp ->
        resp.Response.message.content
        |> printfn "%s"
    | _ -> ()

ℹ INFO: 
EndPoint: http://localhost:11434/api/chat
Payload:
{"messages":[{"content":"What is the weather in Seattle?","role":"user"}],"model":"joefamous/firefunction-v1:q3_k","options":{"num_keep":null,"seed":101,"num_predict":null,"top_k":null,"top_p":null,"tfs_z":null,"typical_p":null,"repeat_last_n":64,"temperature":0.0,"repeat_penalty":null,"presence_penalty":null,"frequency_penalty":null,"mirostat":0,"mirostat_tau":null,"mirostat_eta":null,"penalize_newline":null,"stop":[],"numa":null,"num_ctx":2048,"num_batch":null,"num_gqa":null,"num_gpu":null,"main_gpu":null,"low_vram":null,"f16_kv":null,"vocab_only":null,"use_mmap":null,"use_mlock":null,"rope_frequency_base":null,"rope_frequency_scale":null,"num_thread":null},"stream":false,"tools":[{"type":"function","function":{"name":"get_weather","description":"Get the weather.","parameters":{"type":"object","properties":{"city":{"description":"The city to get the weather for","type":"string"},"state":{"description":"The state to get the w

Error: System.Threading.Tasks.TaskCanceledException: A task was canceled.
   at Microsoft.FSharp.Control.AsyncResult`1.Commit() in D:\a\_work\1\s\src\FSharp.Core\async.fs:line 454
   at Microsoft.FSharp.Control.AsyncPrimitives.RunImmediate[a](CancellationToken cancellationToken, FSharpAsync`1 computation) in D:\a\_work\1\s\src\FSharp.Core\async.fs:line 1160
   at Microsoft.FSharp.Control.AsyncPrimitives.RunSynchronously[T](CancellationToken cancellationToken, FSharpAsync`1 computation, FSharpOption`1 timeout) in D:\a\_work\1\s\src\FSharp.Core\async.fs:line 1166
   at Microsoft.FSharp.Control.FSharpAsync.RunSynchronously[T](FSharpAsync`1 computation, FSharpOption`1 timeout, FSharpOption`1 cancellationToken) in D:\a\_work\1\s\src\FSharp.Core\async.fs:line 1511
   at <StartupCode$FSI_0010>.$FSI_0010.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodBaseInvoker.InvokeWithNoArgs(Object o